In [ ]:
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np 
import sqlite3
from itertools import product
import pathlib

Some notebook and data-specific configuration values for plotting.

In [ ]:
plt.rcParams['text.usetex'] = True
fig_output_dir="Figures"
pathlib.Path(fig_output_dir).mkdir(parents=True, exist_ok=True) 

data_src_dir = "ProcOutputs"
qubit_count = {"RX" : 1, "H" : 1, "CNOT" : 2}
lightning_kernels = ["LM", "AVX2", "AVX512"]

name_map = {"lightning" : "LightningQubit","lightningStream" : "LightningQubitStream", "qulacs" : "Qulacs", "intelqs":"IntelQS", "quest": "QuEST"}
devices = list(name_map.keys())
omp_values = [1,4,16,32]
cmap = matplotlib.colormaps['tab20c']
colors = {k : cmap.colors[1+4*l] for k,l in zip(devices, range(len(devices)))}

## Data-loader utilities functions

In [ ]:
def load_data(gate: str):
    "Convert CSV data with added header to accessible dictionary"
    num_q = qubit_count[gate]
    if num_q == 1:
        filename = 'ProcOutputs/header_1q.csv'
    else:
        filename = 'ProcOutputs/header_2q.csv'

    with open(filename, 'r') as h:
        header = h.read()
    col_names = header[0:-1].split(",")
    df_others = pd.read_csv(f"{data_src_dir}/output_{gate}.csv", names=col_names) # All other frameworks
    df_lq_omp = pd.read_csv(f"{data_src_dir}/output_{gate}_lqomp.csv", names=col_names) # OpenMP accelerated Lightning
    df_lq_omp_s = pd.read_csv(f"{data_src_dir}/output_{gate}_lqompstream.csv", names=col_names) # OpenMP accelerated Lightning

    data = {}
    data["LightningOMPBin"] = df_lq_omp
    data["LightningOMPStreamBin"] = df_lq_omp_s
    labels = set(df_others["sim"])
    
    for l in labels:
        data[l] = df_others[df_others["sim"] == l]
    return data

In [ ]:
def db_load(gate = "CNOT", qubits = 30, omp = 1, sim = "lightning", kernel = 'LM'):
    "Load data through SQL intermediary to subselect the required data for plotting"
    data = load_data(gate)
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()

    data["LightningOMPBin"].to_sql("lightning", conn)
    data["LightningOMPStreamBin"].to_sql("lightningStream", conn)
    data["QulacsBin"].to_sql("qulacs", conn)
    data["IQSBin"].to_sql("intelqs", conn)
    data["QuESTBin"].to_sql("quest", conn)
    if gate in ("H", "RX"):
        if sim == "lightning":
            cursor.execute(f'''SELECT tgt, t_avg FROM {sim} WHERE qubits=? AND omp=? AND (kernel="{kernel}" OR kernel IS NULL)''', (qubits, omp))
        else:
            cursor.execute(f'''SELECT tgt, t_avg FROM {sim} WHERE qubits=? AND omp=?''', (qubits, omp))
    else:
        if sim == "lightning":
            cursor.execute(f'''SELECT ctrl, tgt, t_avg FROM {sim} WHERE qubits=? AND omp=? AND (kernel="{kernel}" OR kernel IS NULL)''', (qubits, omp))
        else:
            cursor.execute(f'''SELECT ctrl, tgt, t_avg FROM {sim} WHERE qubits=? AND omp=?''', (qubits, omp))

    all_rows = cursor.fetchall()
    data_out = []
    for row in all_rows:
        # row[0] returns the first column in the query (name), row[1] returns email column.
        if not gate in ("H", "RX"):
            data_out.append([row[0], row[1], row[2]])
        else:
            data_out.append([row[0], row[1]])

    return data_out


## Plotting functions

In [ ]:
def plot_gate_multisim_1q(gate = "H", qubits=30, optional_kernel = "AVX512"):
    "Plot simulators for a given gate grouped by OpenMP threads"
    plt.rcParams.update({'font.size': 16})

    for omp in omp_values:
        f = plt.figure()
        x = range(qubits)
        for j in devices:
            y = [i[qubit_count[gate]]*1e-3 for i in db_load(gate=gate, qubits = qubits, omp = omp, sim = j, kernel=optional_kernel)]
            plt.plot(x, y,
                     label=f"$\\textrm{{{name_map[j]}}}$", marker="o", 
                     linewidth=2, mec='black', mew=1,
                    )
        plt.xlabel("$\\textrm{gate index}$")
        plt.ylabel("$t$ (s)")
        plt.legend(ncol=2, fontsize=12, fancybox=True, shadow=True)
        plt.tight_layout()
        plt.savefig(f"{fig_output_dir}/{gate}_Lineplots_OMP{omp}_qubits{qubits}.pdf", dpi=200)
        plt.close(f)

In [ ]:
def plot_gate_bar(gate = "H", qubits=30, optional_kernel = "AVX512", use_semilogy=False):    
    "Bar plot of averaged gate runtiomes over all indices grouped by OpenMP threading"
    plt.rcParams.update({'font.size': 16})
    f = plt.figure()
    if use_semilogy:
        plt.semilogy()
    offsets = np.linspace(-0.25,0.25, len(name_map))
    for omp_idx,omp in enumerate(omp_values):
        for dev_idx,j in enumerate(devices):
            x = omp_idx+offsets[dev_idx]
            data = [i[qubit_count[gate]]*(1/1000) for i in db_load(gate=gate, qubits = qubits, omp = omp, sim = j, kernel="AVX512")]
            data_m, data_std = np.mean(data), np.std(data)
            if omp_idx == 0:
                label = f"$\\textrm{{{name_map[j]}}}$"
            else:
                label = None
            plt.bar(x, data_m, label=label, color=colors[j], width=(offsets[1] - offsets[0]),
                     linewidth=2, ec='black',
                    )
            plt.errorbar(x, data_m, yerr=data_std, color="k", linewidth=2, capthick=1, fcolor=None, fmt='.')
    plt.xticks(range(4), [f"${{{i}}}$" for i in omp_values])
    plt.xlabel("$\\textrm{OpenMP threads}$")
    plt.ylabel("$t~\\textrm{(s)}$")
    plt.yscale('log')

    plt.legend(ncol=1, fontsize=12, fancybox=True, shadow=True)
    plt.tight_layout()
    plt.savefig(f"{fig_output_dir}/{gate}_{qubits}_bar_tavg_vs_omp.pdf", dpi=200)
    plt.close(f)

In [ ]:
def plot_gate_1q_kernels(gate = "RX", qubits=30):
    "Line-plots of LightningQubit gates for kernels vs OpenMP threads"
    fig, axs = plt.subplots(2, 2, sharex=True)
    plt.rcParams.update({'font.size': 12})

    x = range(qubits)
    for idx,omp in enumerate(omp_values):
        for j in lightning_kernels:
            y = [i[qubit_count[gate]]*1e-3 for i in db_load(gate=gate, qubits = qubits, omp = omp, sim = "lightning", kernel=j)]
            axs[idx%2, idx//2].plot(x, y, 
                     label=f"$\\textrm{{{j}}}$", marker="o", 
                     linewidth=2, mec='black', mew=1,
                    )
        if idx%2:
            axs[idx%2, idx//2].set_xlabel("$\\textrm{gate index}$", fontsize=16)
        if not idx//2:
            axs[idx%2, idx//2].set_ylabel("$t~\\textrm{(s)}$", fontsize=16)

        if omp == 32:
            axs[1,1].legend(loc='lower left', bbox_to_anchor=(-0.95, -0.6), ncol=3, fancybox=True, shadow=True)

        axs[idx%2, idx//2].set_title(f"$\\textrm{{OMP_NUM_THREADS}}={omp}$", fontsize=12)
        plt.tight_layout()
    plt.subplots_adjust(wspace=0.35, hspace=0.25)
    plt.savefig(f"{fig_output_dir}/{gate}_q{qubits}_Lineplots_LQ_Kernels.pdf", dpi=200)
    plt.close(fig)

## Generate all figures

In [ ]:
for gate,qubits in product(["RX", "H"], [30]):
    plot_gate_multisim_1q(gate, qubits)
    plot_gate_1q_kernels(gate, qubits)
    plot_gate_bar(gate, qubits)

for qubits in [30]:
    plot_gate_bar("CNOT", qubits)

In [ ]:
def plot_gate_bar_single(qubits=30, optional_kernel = "AVX512", use_semilogy=True):    
    "Bar plot of averaged gate runtiomes over all indices grouped by OpenMP threading with gates combined into a single pdf"
    plt.rcParams.update({'font.size': 16})
    f, axs = plt.subplots(3,1, sharex=True, squeeze=True)

    offsets = np.linspace(-0.25,0.25, len(name_map))

    ylims = [(1e-1, 2.5), (1e-1, 2.5), (5e-2, 2.5)]
    
    for g_idx,gate in enumerate(["H", "RX", "CNOT"]):
        ax = axs[g_idx]
        ax.set_ylim(ylims[g_idx])
        if use_semilogy:
            ax.set_yscale('log')
    
        for omp_idx,omp in enumerate(omp_values):        
            for dev_idx,j in enumerate(devices):
    
                x = omp_idx+offsets[dev_idx]
                data = [i[qubit_count[gate]]*(1/1000) for i in db_load(gate=gate, qubits = qubits, omp = omp, sim = j, kernel="AVX512")]
                data_m, data_std = np.mean(data), np.std(data)
                if omp_idx == 0:
                    if name_map[j] == "LightningQubitStream":
                        name_map[j] = "LightningQubit (stream)"
                    label = f"$\\textrm{{{name_map[j]}}}$"
                else:
                    label = None
                ax.bar(x, data_m, label=label, color=colors[j], width=(offsets[1] - offsets[0]),
                         linewidth=2, ec='black',
                        )
                ax.errorbar(x, data_m, yerr=data_std, color="k", linewidth=2, capthick=1, fcolor=None, fmt='.')
        ax.set_xticks(range(4), [f"${{{i}}}$" for i in omp_values])

    axs[1].set_ylabel("$t~\\textrm{(s)}$")
    axs[-1].set_xlabel("$\\textrm{OpenMP threads}$")
    
    axs[0].legend(loc='best', fontsize=12, ncol=2, fancybox=True, shadow=True)
    plt.subplots_adjust(wspace=0.35, hspace=0.25)


Manually adjust figure sizing after plotting and save.

In [ ]:
plot_gate_bar_single()
plt.subplots_adjust(wspace=0.1, hspace=0.1)

In [ ]:
plt.tight_layout()

In [ ]:
plt.savefig(f"{fig_output_dir}/H_RX_CNOT_30_bar_tavg_vs_omp_s.pdf", dpi=200)

In [ ]:
plt.subplots_adjust(
    left  = 0.125,  # the left side of the subplots of the figure
    right = 0.9,    # the right side of the subplots of the figure
    bottom = 0.1,   # the bottom of the subplots of the figure
    top = 0.9,      # the top of the subplots of the figure
    wspace = 0.2,   # the amount of width reserved for blank space between subplots
    hspace = 0.2,   # the amount of height reserved for white space between subplots
)

In [ ]:
plt.subplots_adjust(wspace=0.1, hspace=0.05)

The above utility demonstrates the perf change given by the streaming operations.

In [ ]:
def streaming_rel(gate = "CNOT", omp = 32):
    data_lqs_m = np.mean(db_load(gate=gate, qubits = 30, omp = omp, sim = "lightningStream", kernel="AVX512"))
    data_lq_m = np.mean(db_load(gate=gate, qubits = 30, omp = omp, sim = "lightning", kernel="AVX512"))
    data_qu_m = np.mean(db_load(gate=gate, qubits = 30, omp = omp, sim = "qulacs",))
    data_iqs_m = np.mean(db_load(gate=gate, qubits = 30, omp = omp, sim = "intelqs",))
    data_quest_m = np.mean(db_load(gate=gate, qubits = 30, omp = omp, sim = "quest",))

    
    return {"lightning" : (data_lq_m)/data_lqs_m, "qulacs" : data_qu_m/data_lqs_m, "intelqs" : data_iqs_m/data_lqs_m, "data_quest_m": data_quest_m/data_lqs_m}


In [ ]:
streaming_rel("CNOT", omp=16)

The folloing generate the gate-level LQ data with the streaming operations included for comparison.

In [ ]:
def plot_gate_1q_kernels_streaming(gate = "H", qubits=30):
    "Line-plots of LightningQubit gates for kernels vs OpenMP threads"
    fig, axs = plt.subplots(2, 2, sharex=True)
    plt.rcParams.update({'font.size': 12})
    #colors = [cmap.colors[1+4*i] for i in range(4)]

    x = range(qubits)
    for idx,omp in enumerate(omp_values):
        for jdx,j in enumerate(lightning_kernels):
            y = [i[qubit_count[gate]]*1e-3 for i in db_load(gate=gate, qubits = qubits, omp = omp, sim = "lightning", kernel=j)]
            axs[idx%2, idx//2].plot(x, y, 
                     label=f"$\\textrm{{{j}}}$", #marker="o", 
                     linewidth=3, #mec='black', mew=1, #color=colors[jdx]
                    )
        y = [i[qubit_count[gate]]*1e-3 for i in db_load(gate=gate, qubits = qubits, omp = omp, sim = "lightningStream", kernel="AVX512")]
        axs[idx%2, idx//2].plot(x, y, 
                     label="$\\textrm{AVX512 (stream)}$", #marker="o", 
                     linewidth=3, #mec='black', mew=1,# color=colors[3]
                    )
        if idx%2:
            axs[idx%2, idx//2].set_xlabel("$\\textrm{gate index}$", fontsize=16)
        if not idx//2:
            axs[idx%2, idx//2].set_ylabel("$t~\\textrm{(s)}$", fontsize=16)

        if omp == 32:
            axs[1,1].legend(loc='lower left', bbox_to_anchor=(-0.8, -0.65), ncol=2, fancybox=True, shadow=True)

        axs[idx%2, idx//2].set_title(f"$\\textrm{{OMP_NUM_THREADS}}={omp}$", fontsize=12)
        plt.tight_layout()
    plt.subplots_adjust(wspace=0.35, hspace=0.25)
    #plt.savefig(f"{fig_output_dir}/{gate}_q{qubits}_Lineplots_LQ_Kernels.pdf", dpi=200)
    #plt.close(fig)

In [ ]:
plot_gate_1q_kernels_streaming("RX")

In [ ]:
plt.subplots_adjust(wspace=0.2, hspace=0.3)

In [ ]:
gate="RX"
qubits=30
plt.savefig(f"{fig_output_dir}/{gate}_q{qubits}_Lineplots_LQ_Kernels_Stream.pdf", dpi=200)

In [ ]:
plt.tight_layout()